# Konvolučné neurónové siete - LeNet

Na dnešnom cvičení naimplementujeme jeden zo základných konvolučných neurónových sietí, konkrétne LeNet. Naše riešenie budeme aplikovať na rozpoznávanie rukou písaných číslic z datasetu MNIST. LeNet bol pôvodne navrhnutý v roku 1989 pre rozpoznávanie číslic amerických PSČ. Sieť následne prešla niekoľkými iteráciami a v roku 1998 bol [publikovaný model lenet-5](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=726791), ktorý bol presnejší ako bežné modely počítačového videnia na rozpoznávanie rukou písaných číslic. LeNet bol takisto medzi prvými konvolučnými sieťami, ktoré používali algoritmus spätného šírenia chyby na trénovanie. V našom riešení urobíme niekoľko zmien oproti pôvodnému článku, najmä čo sa týka použitej aktivačnej funkcie (pôvodne `tanh`) a optimalizátora (pôvodne jednoduchý `SGD`).

Predpokladom na úspešné zvládnutie tohto cvičenia je pripravené programátorské prostredie s `tensorflow`om a `keras`om. Ak dané nástroje ste si ešte nenainštalovali, urobte tak podľa [tohto návodu](https://github.com/ianmagyar/dl-course/blob/master/labs/00%20-%20Setting-up-tensorflow.md). V priebehu cvičenia sa takisto oboznámime so základnými krokmi pri trénovaní hlbokých neurónových sietí.

Kostru riešenia nájdete [tu](resources/lab04/lab04.py) alebo môžete pracovať priamo v tomto jupyter notebooku.

## 1. Načítanie potrebných knižníc

V prvom kroku načítame všetky potrebné moduly pre definovanie, trénovanie a vyhodnotenie siete. Dokumentáciu príslušných tried a metód nájdete [tu pre keras](https://keras.io/api/) a [tu pre scikit-learn](https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics).

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.optimizers import Adam

from keras.datasets import mnist
from keras.utils import to_categorical

from sklearn.metrics import classification_report, confusion_matrix

## 2. Predspracovanie údajov

Prvým dôležitým krokom pri vývoja hlbokých riešení je načítanie a predspracovanie datasetu. Samotné predpsracovanie v sebe zahŕňa hneď niekoľko úloh, ako výber príznakov, normalizácia hodnôt, vektorizácia vstupov a výstupov alebo rozdelenie datasetu na trénovaciu a testovaciu (prípadne validačnú) množinu.

V skripte nájdete príkaz na načítanie datasetu MNIST, ktorý obsahuje čiernobiele obrázky rukou písaných číslic (viď nižšie) od 0 po 9. Dataset vieme načítať priamo z knižnice `keras` cez funkciu `load_data`. Pomocné premenné `img_height` a `img_width` obsahujú rozmery každého obrázka a slúžia pre opätovné použitie týchto hodnôt neskôr v kóde.

![MNIST dataset](https://miro.medium.com/max/495/0*94t_5cPF9mvBj20z.png)

In [ ]:
# we load the dataset from keras
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# we set the image sizes for further use
img_height, img_width = 28, 28

Ako vidíte v kóde, pri načítaní datasetu hneď ho rozdelíme aj na trénovaciu a testovaciu množinu.

Ďalším krokom je normalizácia hodnôt. V prvom rade potrebujeme vstupné čísla reprezentujúce pixely normalizovať na interval 0-1 a ešte je potrebné upraviť dimenzionalitu `numpy` polí, ktoré obsahujú vstupné dáta. Polia `x_train` a `x_test` rozšírime o jednu dimenziu s rozmerom 1, teda každú hodnotu "zabalíme" do wrappera, aby sa nám lepšie pracovalo s nimi.

In [ ]:
# reshape the training and testing input sets - add a channel
# normalize input data to interval 0 - 1

Výstupné dáta `y_train` a `y_test` majú formu poľa celých čísel od 0 po 9 (reprezentujúce číslice), keďže ale neriešime regresiu ale klasifikáciu, potrebujeme ich pretransformovať do podoby vektorov. Táto vektorová reprezentácia sa používa veľmi často pri spracovaní kategórií a reťazcov neurónovými sieťami. Na programovú realizáciu existuje niekoľko možností, my dnes použijeme funkciu `to_categorical` z knižnice `keras` ([dokumentácia](https://keras.io/api/utils/python_utils/#to_categorical-function)).

In [ ]:
# normalize (vectorize) output data

**Poznámka:** v niektorých prípadoch dokážete reťazce nahradiť jednoduchými číslami, takýto spôsob ale predpokladá, že čísla, ktoré sú blízko sebe vyjadrujú koncepty, ktoré sú veľmi podobné. Napríklad, ak máme stĺpec s hodnotami *low*, *middle*, *high*, tieto hodnoty vieme nahradiť číslami 1, 2 a 3. Rovnaký spôsob ale nemôžeme použiť s hodnotami napríklad značky auta: *Škoda* (1), *Audi* (2), *Lada* (3), pretože neurónová sieť by predpokladala, že Lada (3) je viac podobná Audi (2) ako Škodovke (1).

## 3. Definícia modelu

Našu hlbokú sieť zadefinujeme pomocou knižnice `keras`, ktorá ponúka vysokoúrovňové API na prácu s neurónovými sieťami. Na vytvorenie modelov máme dve možnosti: najprv vytvoríme objekt modelu a postupne doňho pridávame vrstvy, alebo najprv zadefinujeme a pospájame vrstvy a následne zadefinujeme model pomocou vstupnej a výstupnej vrstvy. Na cvičení použijeme prvý spôsob, sekvenčný model už máte v kóde vytvorený ([dokumentácia](https://keras.io/api/models/sequential/#sequential-class)). Pridajte do tohto modelu vrstvy podľa diagramu nižšie. V konvolučných a plne prepojených vrstvách použite aktiváciu `relu` a vo výstupnej vrstve `softmax`. Konvolučné kernely majú rozmer *5x5*, *pooling* kernely *2x2*.

![](resources/lab04/lenet-structure.png)

In [ ]:
# define model, add layers based on the diagram
model = Sequential()

Vašu implementáciu si môžete skontrolovať pomocou funkcie `summary` ([dokumentácia](https://keras.io/api/models/model/#summary-method)).

In [ ]:
# check the model structure
model.summary()

Veľkým rozdielom medzi `tensorflow`om a `pycharm`om je potreba modely skompilovať pred trénovaním. Slúži na to metóda `compile` ([dokumentácia](https://keras.io/api/models/model_training_apis/#compile-method)). Pomocou nej **skompilujte model zadaním parametrov nasledovne**:

- chybová funkcia - *categorical crossentropy*
- optimalizátor - *Adam*
- metriky - presnosť (*accuracy*)

In [ ]:
# compile the model
model.compile()

## 4. Trénovanie modelu

Po skompilovaní modelu ho môžeme natrénovať, na čo slúži metóda `fit` ([dokumentácia](https://keras.io/api/models/model_training_apis/#fit-method)). Jej hlavné parametre sú:

- trénovací vstup
- trénovací výstup
- `epochs` - počet epoch
- `batch_size` - počet príkladov v jednej dávke (pred aktualizáciou parametrov).

In [ ]:
# train the network
model.fit()

## 5. Vyhodnotenie siete

Vyhodnotenie siete pozostáva z dvoch základných úloh: testovanie a vyhodnotenie. Pre testovanie musíme získať výstupy k vstupným hodnotám z trénovacej množiny pomocou metódy `predict` ([dokumentácia](https://keras.io/api/models/model_training_apis/#predict-method)). Alternatívne môžete na jednoduché vyhodnotenie použiť metódu `evaluate` ([dokumentácia](https://keras.io/api/models/model_training_apis/#evaluate-method)).

In [ ]:
y_pred = model.predict(x_test)

Ďalej porovnáme ozajstné výstupy s očakávanými. Keďže výstup má vektorovú reprezentáciu, potrebujeme zistiť pozíciu kde sa nachádza najväčšia hodnota vo vektore. V tomto nám pomôže knižnica `numpy`, ktorú sme zatiaľ nepoužili explicitne, ale podporuje všetky už použité knižnice. Jedná sa o efektívne a optimalizované riešenie práce s poľami.

Pre vyhodnotenie našej siete použijeme konfúznu maticu. Konfúzna matica je tabuľková reprezentácia, kde v riadkoch máme očakávané triedy a v stĺpcoch vypočítané (predikované). V bunkách tabuľky sú uložené počty príkladov klasifikované v danej kombinácii očakávanej a predikovanej triedy. Ideálny klasifikátor bude mať všetky hodnoty po hlavnej diagonále (ďalšie informácie nájdete na [wikipédii](https://en.wikipedia.org/wiki/Confusion_matrix)).

In [ ]:
y_test_class = np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)

print(confusion_matrix(y_test_class, y_pred_class))

Z konfúznej matici potom vieme vypočítať ďalšie metriky, ako presnosť (*accuracy*), návratnosť (*recall*) a precizita (*precision*):

In [ ]:
print(classification_report(y_test_class, y_pred_class))

Presnosť popisuje samotný klasifikátor a vypočíta sa nasledovne:

$ACC = \frac{TP + TN}{P + N}$

kde $TP + TN$ je suma správne klasifikovaných príkladov (na hlavnej diagonále) a $P + N$ je počet všetkých príkladov.

Návratnosť a precizita popisujú klasifikátor pre danú triedu, vypočítajú sa nasledovne:

$REC = \frac{TP}{P}$

$PREC = \frac{TP}{TP + FP}$

kde $TP$ je počet správne klasifikovaných príkladov z danej triedy, $P$ je počet príkadov z danej triedy v testovacej množine a $FP$ je počet príkladov z testovacej množiny nesprávne klasifikovaných do tejto triedy.

Metóda `classification_report` vypočíta ešte hodnotu F1, ktorá je harmonický priemer návratnosti a precizity:

$F1 = 2 \cdot \frac{REC \cdot PREC}{REC + PREC}$

## Doplňujúce úlohy

1. Originálny model LeNet používal aktivačnú funkciu `tanh` a trénovanie pomocou jednoduchého backpropagation (*stochastic gradient descent*). Upravte váš model a porovnajte presnosť dvoch modelov.
2. Pri trénovaní sa často používa aj validačná množina, ktorá slúži na určenie včasného ukončenia trénovania a aby sme tak predišli pretrénovaniu modelu. Rozšírte vaše riešenie o použitie validačnej množiny.

## Použité zdroje

- [LeCun, Yann, Léon Bottou, Yoshua Bengio, and Patrick Haffner. "Gradient-based learning applied to document recognition." Proceedings of the IEEE 86, no. 11 (1998): 2278-2324.](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=726791&tag=1)
- [LeNet on MNIST with Keras and Tensorflow in Python](https://github.com/matthewrenze/lenet-on-mnist-with-keras-and-tensorflow-in-python)
- [TensorSpace Playground - LeNet](https://tensorspace.org/html/playground/lenet.html)
- [Ukážka vizualizácie konvolučných sietí](lab04b-cnn-visualization.ipynb)